In [8]:
import os

import pandas as pd

from dotenv import load_dotenv
import openai

import datetime

In [11]:
load_dotenv()

openai.organization = os.environ['OPENAI_ORG']
openai.api_key = os.environ['OPENAI_API_KEY']

In [14]:
from transformers import GPT2Tokenizer


TOKENIZER = GPT2Tokenizer.from_pretrained("gpt2")


def get_number_of_tokens(text: str) -> int:
    output = TOKENIZER(text)
    return len(output['input_ids'])

c:\Users\ssds6\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ssds6\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ssds6\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see ac

In [15]:
def get_gpt3_completion(prompt, engine="davinci", **kwargs):
    engines = {
        'davinci': 'text-davinci-002',
        'curie': 'text-curie-001',
        'babbage': 'text-babbage-001',
        'ada': 'text-ada-001'
    }
    
    engine = engines.get(engine, engine)
    
    default = dict(
        max_tokens=3500 if engine == 'text-davinci-002' else 1500,
        stop=" END" if engine not in engines else None
    )
    
    final_kwargs = {**default, **kwargs}
    response = openai.Completion.create(
        prompt=prompt,
        engine=engine,
        **final_kwargs
    )
    
    answer = response['choices'][0]['text']
    return answer

In [20]:
#Load Datos tabulares
ids = 2517004 #2516982 #2516941
path = fr"C:\Users\ssds6\OneDrive\Escritorio\analisis_deportivo\data\fineTuning\{ids}"
metricas = pd.read_excel(os.path.join(path,"playerSeasonInfo.xlsx"))
tabla = pd.read_excel(os.path.join(path,"Tabla.xlsx"))
tabla['Posicion'] = tabla.index +1
partido = pd.read_excel(os.path.join(path,"matchInfo.xlsx"))

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ssds6\\OneDrive\\Escritorio\\analisis_deportivo\\data\\fineTuning\\2517004\\playerSeasonInfo.xlsx'

In [ ]:
metricas.head()

,Unnamed: 0,shortName,currentTeamId,goals,goal_assisted,matches,red_card,yellow_card
0,8,N. \u015eahin,Borussia Dortmund,2,2,17,0,2
1,18,Pablo \u00cdnsua,Schalke 04,0,0,1,0,1
2,23,F. Di Santo,Schalke 04,3,1,27,0,2
3,26,S. Papastathopoulos,Borussia Dortmund,2,0,27,1,3
4,27,Naldo,Schalke 04,7,2,30,0,4


In [ ]:
partido

,Unnamed: 0,index,local,visita,Jornada,Fecha,venue,competitionId,coachIdLocal,coachIdVisita
0,0,38,Schalke 04,Borussia Dortmund,30,2018-04-15 13:30:00,VELTINS-Arena,German first division,D. Tedesco,NaN


In [ ]:
tabla.head()

,Unnamed: 0,victorias,empates,derrotas,gf,ga,gd,Puntos,Posicion
0,Bayern M\u00fcnchen,24,3,3,81,22,59,75,1
1,Schalke 04,16,7,7,47,33,14,55,2
2,Bayer Leverkusen,14,9,7,55,37,18,51,3
3,Borussia Dortmund,14,9,7,57,41,16,51,4
4,Hoffenheim,12,10,8,55,42,13,46,5


In [ ]:
#Extraer informacion
# Partido
matchInfo ={
'Home Team' : partido.iloc[0]['local'],
'Away Team' : partido.iloc[0]['visita'],
'Gameweek' : partido.iloc[0]['Jornada'],
'Venue' : partido.iloc[0]['venue'],
'Competition' : "Bundesliga",
'HomeCoach' : partido.iloc[0]['coachIdLocal'],
'AwayCoach' : partido.iloc[0]['coachIdVisita']}

Dateutc = datetime.datetime.strptime(partido.iloc[0]['Fecha'],'%Y-%m-%d %H:%M:%S')
matchInfo['Date'] = str(Dateutc.date())
matchInfo['Time UTC'] = str(Dateutc.time())

#Torneo
homeTable = tabla[tabla['Unnamed: 0']==HomeTeam].reset_index()

homeTable = {
'Home Position' : homeTable.iloc[0]['Posicion'],
'Home Points' : homeTable.iloc[0]['Puntos'],
'Home Wins' : homeTable.iloc[0]['victorias'],
'Home Ties' : homeTable.iloc[0]['empates'],
'Home Defeats' : homeTable.iloc[0]['derrotas'],
'Home Goals Scored':homeTable.iloc[0]['gf'],
'Home Goals Against':homeTable.iloc[0]['ga']}

idx = tabla.index.get_loc(homeTable['Home Position']-1)
test = tabla.iloc[idx - 1 : idx + 2].reset_index()

homeTable['Team Above Home Team'] = test.iloc[0]['Unnamed: 0']
homeTable['Points Team Above Home Team'] = test.iloc[0]['Puntos']

homeTable['Team Below Home Team'] = test.iloc[2]['Unnamed: 0']
homeTable['Points Team Below Home Team'] = test.iloc[2]['Puntos']

awayTable = tabla[tabla['Unnamed: 0']==AwayTeam].reset_index()

awayTable = {
'Away Position' : awayTable.iloc[0]['Posicion'],
'Away Points' : awayTable.iloc[0]['Puntos'],
'Away Wins' : awayTable.iloc[0]['victorias'],
'Away Ties' : awayTable.iloc[0]['empates'],
'Away Defeats' : awayTable.iloc[0]['derrotas'],
'Away Goals Scored':awayTable.iloc[0]['gf'],
'Away Goals Against':awayTable.iloc[0]['ga']}

idx = tabla.index.get_loc(awayTable['Away Position']-1)
test = tabla.iloc[idx - 1 : idx + 2].reset_index()

awayTable['Team Above Away Team'] = test.iloc[0]['Unnamed: 0']
awayTable['Points Team Above Away Team'] = test.iloc[0]['Puntos']

awayTable['Team Below Away Team'] = test.iloc[2]['Unnamed: 0']
awayTable['Points Team Below Away Team'] = test.iloc[2]['Puntos']

#Metricas
#Home
homeStats = {}
metricasHome = metricas[metricas['currentTeamId'] == HomeTeam]
metricasHome = metricasHome.sort_values(by='goals', ascending=False).reset_index()
homeStats['Home Top Scorer'] = metricasHome.iloc[0]['Unnamed: 0']
metricasHome = metricasHome.sort_values(by='goal_assisted',ascending=False).reset_index()
homeStats['Home Top Assister'] = metricasHome.iloc[0]['Unnamed: 0']
metricasHome = metricasHome.sort_values(by='red_card',ascending=False).reset_index(drop=True)
homeStats['Home Most Sent Off'] = metricasHome.iloc[0]['Unnamed: 0']
#Away
awayStats = {}
metricasAway = metricas[metricas['currentTeamId'] == AwayTeam]
metricasAway = metricasAway.sort_values(by='goals', ascending=False).reset_index()
awayStats['Away Top Scorer'] = metricasAway.iloc[0]['Unnamed: 0']
metricasAway = metricasAway.sort_values(by='goal_assisted', ascending=False).reset_index()
awayStats['Away Top Assister'] = metricasAway.iloc[0]['Unnamed: 0']
metricasAway = metricasAway.sort_values(by='red_card', ascending=False).reset_index(drop=True)
awayStats['Away Most SentOff'] = metricasAway.iloc[0]['Unnamed: 0']

In [ ]:
#Escribir datos para prompt
separador=' : '

match_prompt = ''

for k,v in matchInfo.items():
    aux = k + separador + str(v) + ' ; '
    match_prompt += aux

home_team_prompt = ''
home_stats = ''
for k,v in homeStats.items():
    aux = k + separador + str(v) + ' ; '
    home_stats += aux

home_table = ''
for k,v in homeTable.items():
    aux = k + separador + str(v) + ' ; '
    home_table += aux
    
home_team_prompt = 'The individual stats info on the home team is: ' + home_stats + ' The position info on the home team is: ' + home_table
home_team_prompt

'The individual stats info on the home team is: Home Top Scorer : 247 ; Home Top Assister : 92 ; Home Most Sent Off : 92 ;  The position info on the home team is: Home Position : 2 ; Home Points : 55 ; Home Wins : 16 ; Home Ties : 7 ; Home Defeats : 7 ; Home Goals Scored : 47 ; Home Goals Against : 33 ; Team Above Home Team : Bayern M\\u00fcnchen ; Points Team Above Home Team : 75 ; Team Below Home Team : Bayer Leverkusen ; Points Team Below Home Team : 51 ; '

None


In [ ]:
metricas.columns

Index(['Unnamed: 0', 'shortName', 'currentTeamId', 'goals', 'goal_assisted',
       'matches', 'red_card', 'yellow_card'],
      dtype='object')

In [ ]:
prompt = """
"""

comp = get_gpt3_completion(prompt)

In [ ]:
comp